In [1]:
import pandas as pd
import os
import sys

mod = sys.modules[__name__]
def excel_reader():
    # 해당 경로에 있는 .xlsx 포맷의 파일이름을 리스트로 가져오기
    path = "/home/u1026/tm_2020/"
    file_list = os.listdir(path)
    exfile_list = [i for i in file_list if os.path.splitext(i)[-1]==".xlsx"]

    # 파일이름을 split 해 해당 파일의 날짜 리스트로 가져오기
    fname_list = []
    for i in range(len(exfile_list)):
        fname_list += [exfile_list[i].split("-")[0].split("_")[1]]

    # exfile_list에 있는 엑셀파일 모두 읽어오기
    for k in range(len(exfile_list)):
        file_locate = path + exfile_list[k]
        setattr(mod, 'file_{}'.format(fname_list[k]), pd.read_excel(file_locate))

In [2]:
excel_reader()

In [3]:
import pandas as pd

columns_name = ['일자', '제목', '본문', '인물', '기관', '키워드', '특성추출(가중치순 상위 50개)', 'URL']
file_20200801 = file_20200801[columns_name]
file_20200802 = file_20200802[columns_name]

news_20200801 = file_20200801
news_20200802 = file_20200802

#'제목'열에서 na가 있으면 해당 행 삭제
news_20200801 = news_20200801.dropna(subset=['제목'])
news_20200802 = news_20200802.dropna(subset=['제목'])

#row번호 리셋하기
news_20200801 = news_20200801.reset_index()
news_20200802 = news_20200802.reset_index()


#제목 마지막에 공백을 넣어 본문과 합칠 때 자연스럽게 합치게 함
for i in range(0,len(news_20200801)):
    news_20200801['제목'][i] = news_20200801['제목'][i] + ' '
    
for i in range(0,len(news_20200802)):
    news_20200802['제목'][i] = news_20200802['제목'][i] + ' '


news_20200801['내용'] = news_20200801.iloc[:, 2:4].sum(1)
news_20200802['내용'] = news_20200802.iloc[:, 2:4].sum(1)

for i in range(0,len(news_20200801)):
    news_20200801['내용'][i] = news_20200801['내용'][i].replace(',','').replace('\n','').replace('.','').replace('"','').replace('!','').replace('(',' ').replace(')','').replace('?','').casefold()
#숫자 지우기
#news_20200801['내용'] = news_20200801['내용'].str.replace('[0-9]', '')

for i in range(0,len(news_20200802)):
    news_20200802['내용'][i] = news_20200802['내용'][i].replace(',','').replace('\n','').replace('.','').replace('"','').replace('!','').replace('(',' ').replace(')','').replace('?','').casefold()
#news_20200802['내용'] = news_20200802['내용'].str.replace('[0-9]', '')

/home/u1026/.local/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/u1026/.local/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/u1026/.local/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/u1026/.local/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopy

In [4]:
#word_tokenize : 단어 기준으로 토큰화(Tokenize)
from nltk.tokenize import word_tokenize

word_20200801 = []
for i in range(len(news_20200801)):
    word_20200801 += (word_20200801, word_tokenize(news_20200801['내용'][i]))

In [5]:
#vocabulary 한글자이면 지우기
for i in word_20200801:
    if len(i) == 1:
        word_20200801.remove(i)

print(len(word_20200801))

5454


In [5]:
from konlpy.tag import Komoran

#메모리 늘려주기
komoran = Komoran(max_heap_size=1024 * 6)

#pos사용으로 수정함
km_word_20200801 = []
for i in range(len(news_20200801)):
    #print(okt.morphs(news_190801['내용'][i]))
    #km_word_20200801 +=  komoran.nouns(news_20200801['내용'][i])
    km_word_20200801 +=  komoran.pos(news_20200801['내용'][i])

In [ ]:
len(km_word_20200801)

In [21]:
komoran.tagset.keys()

dict_keys(['EC', 'EF', 'EP', 'ETM', 'ETN', 'IC', 'JC', 'JKB', 'JKC', 'JKG', 'JKO', 'JKQ', 'JKS', 'JKV', 'JX', 'MAG', 'MAJ', 'MM', 'NA', 'NF', 'NNB', 'NNG', 'NNP', 'NP', 'NR', 'NV', 'SE', 'SF', 'SH', 'SL', 'SN', 'SO', 'SP', 'SS', 'SW', 'VA', 'VCN', 'VCP', 'VV', 'VX', 'XPN', 'XR', 'XSA', 'XSN', 'XSV'])

In [20]:
komoran.tagset.values()

dict_values(['연결 어미', '종결 어미', '선어말어미', '관형형 전성 어미', '명사형 전성 어미', '감탄사', '접속 조사', '부사격 조사', '보격 조사', '관형격 조사', '목적격 조사', '인용격 조사', '주격 조사', '호격 조사', '보조사', '일반 부사', '접속 부사', '관형사', '분석불능범주', '명사추정범주', '의존 명사', '일반 명사', '고유 명사', '대명사', '수사', '용언추정범주', '줄임표', '마침표, 물음표, 느낌표', '한자', '외국어', '숫자', '붙임표(물결,숨김,빠짐)', '쉼표,가운뎃점,콜론,빗금', '따옴표,괄호표,줄표', '기타기호 (논리수학기호,화폐기호)', '형용사', '부정 지정사', '긍정 지정사', '동사', '보조 용언', '체언 접두사', '어근', '형용사 파생 접미사', '명사파생 접미사', '동사 파생 접미사'])

In [24]:
#j로 시작하는 tag들은 모두 조사 - i는 인덱스번호, k는 리스트의 요소
for i, k in enumerate(km_word_20200801):
    #튜플인 k의 1번째 값(예-'JX')에 'J'가 첫번째 글자로 들어가 있으면 조사이므로 지운다.
    if k[1].find('J') == 0:
        km_word_20200801.pop(i)

In [25]:
km_word_20200801[:15]

[('모츠', 'NA'),
 ('motz', 'SL'),
 ('비대', 'NNP'),
 ('이', 'VCP'),
 ('면', 'EC'),
 ('라이브', 'NNP'),
 ('이벤트', 'NNP'),
 ('솔루션', 'NNP'),
 ('출시', 'NNG'),
 ('지금', 'NNG'),
 ('전혀', 'MAG'),
 ('다른', 'MM'),
 ('언', 'NNG'),
 ('택', 'NNG'),
 ('트', 'VV')]

In [26]:
stop_words = []
with open('./data_files/stopwords_list1.txt', 'r') as file1, \
open('./data_files/stopwords_list2.txt', 'r') as file2:
    for text1 in file1:
        stop_words.append(text1.strip('\n'))
    print(len(stop_words))
    for text2_1 in file2:
        text2 = text2_1.split('\t')
        stop_words.append(text2[0].strip('\n'))
    print(len(stop_words))

#불용어리스트 생성
stopwords_set = set(stop_words)
stop_words = list(stopwords_set)


#불용어 제거하기
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

#komoran 사용할 때 - pos사용으로 수정함
word_result_20200801 = []
for w, pos in km_word_20200801:
    if w not in stop_words:
        word_result_20200801.append(w)

print("----불용어 제거 전----")
print(len(km_word_20200801))
print("----불용어 제거 후----")
print(len(word_result_20200801))

675
775
----불용어 제거 전----
236210
----불용어 제거 후----
189554


In [27]:
#vocabulary 만들기
print(len(word_result_20200801))
set_20200801 = set(word_result_20200801)
voca_20200801 = list(set_20200801)
print(len(voca_20200801))

"""
#vocabulary 한글자이면 지우기
for i in voca_20200801:
    if len(i) == 1:
        voca_20200801.remove(i)

print(len(voca_20200801))
"""

189554
16402


'\n#vocabulary 한글자이면 지우기\nfor i in voca_20200801:\n    if len(i) == 1:\n        voca_20200801.remove(i)\n\nprint(len(voca_20200801))\n'

In [28]:
#vocabulary 한글자이면 지우기
for i in voca_20200801:
    if len(i) == 1:
        voca_20200801.remove(i)

print(len(voca_20200801))

15547


In [10]:
voca_20200801[:5]

['존중', '히스패닉', '르마', '구급차', '주리']

In [29]:
#only 숫자인 텍스트 지우기
import re
imsi = []
for tx in voca_20200801:
    hangul = re.compile('[ㄱ-ㅣ가-힣a-zA-Z]+')
    imsi.append(hangul.findall(tx))
VOCA_20200801 = []
for k in range(0, len(voca_20200801)):
    if imsi[k] != []:
        VOCA_20200801.append(voca_20200801[k])

print(len(VOCA_20200801))

14852


In [35]:
len(VOCA_20200801[1])

4

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_20200801 = tfidf.fit_transform(VOCA_20200801)
print(tfidf_20200801[:10])

  (0, 2059)	0.7071067811865475
  (0, 7188)	0.7071067811865475
  (1, 6601)	1.0
  (2, 9811)	1.0
  (3, 4029)	1.0
  (4, 4240)	1.0
  (5, 3724)	1.0
  (6, 83)	0.6712982288428279
  (6, 41)	0.7411873501028484
  (7, 1776)	1.0
  (8, 8295)	1.0
  (9, 11276)	1.0


In [12]:
#DTM
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
dtm_ar_20200801 = cv.fit_transform(VOCA_20200801).toarray()
feature_names = cv.get_feature_names()
#dtm을 데이터프레임 형태로
dtm_20200801 = pd.DataFrame(dtm_ar_20200801, columns=feature_names)

In [13]:
dtm_20200801.shape

(12216, 12072)

In [14]:
import math
N = len(news_20200801)

def tf(token, doc):
    return doc.count(token)

def idf(token):
    df = 0
    for doc in list(news_20200801['내용']):
        df += token in doc
    return math.log(N / (df + 1))

def tf_idf(token, doc):
    return tf(token, doc) * idf(token)

In [30]:
#DTM 만들기
tf_result = []
for i in range(N):
    tf_result.append([])
    doc = news_20200801['내용'][i]
    for j in range(len(VOCA_20200801)):
        token = VOCA_20200801[j]        
        tf_result[-1].append(tf(token, doc))

dtm_20200801 = pd.DataFrame(tf_result, columns = VOCA_20200801)

In [31]:
dtm_20200801.head()

,존중,히스패닉,르마,구급차,주리,장애인,플라이 아웃,증원,도록,이따,...,막국수,이상배,최희,전철,입법예고,그라운드,방송선,작년,신현수,화성
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
dtm_20200801.shape

(2727, 14852)

In [33]:
voca_appear_20200801 = dtm_20200801.sum()
voca_appear_20200801.sort_values(ascending = False)

서       4705
니다      1465
코로      1249
로나      1247
코로나     1245
        ... 
휘말리        0
종규         0
끌리         0
가로막히       0
어울리        0
Length: 14852, dtype: int64